In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler



# Подготовка датасета для загрузки в сеть

In [2]:
base_df = pd.read_csv('data/clean_frame.csv', index_col='ind')
base_df.columns

Index([&#39;ID&#39;, &#39;Код_группы&#39;, &#39;Год_Поступления&#39;, &#39;Год_Окончания_УЗ&#39;, &#39;Пособие&#39;,
       &#39;Общежитие&#39;, &#39;Наличие_Матери&#39;, &#39;Наличие_Отца&#39;, &#39;Опекунство&#39;, &#39;Село&#39;,
       &#39;Иностранец&#39;, &#39;КодФакультета&#39;, &#39;СрБаллАттестата&#39;, &#39;Статус&#39;, &#39;male&#39;,
       &#39;female&#39;, &#39;sex_summ&#39;, &#39;birth_year_int&#39;, &#39;basis&#39;, &#39;language&#39;, &#39;country&#39;,
       &#39;region&#39;, &#39;city&#39;, &#39;parents_country&#39;],
      dtype=&#39;object&#39;)

### Проверка и удаление отрицательных значений

In [7]:
base_df.isna().any().all()

False

In [5]:
base_df = base_df.fillna(0)

In [8]:
columns = ['Наличие_Матери', 'Село', 'Иностранец', 'СрБаллАттестата', 'Статус', 'male', 'basis']
columns = base_df.columns.to_list()
priznaki = len(columns)
priznaki

24

In [10]:
frame_to_train = base_df.loc[:, columns]
frame_to_train.head(2)

,ID,Код_группы,Год_Поступления,Год_Окончания_УЗ,Пособие,Общежитие,Наличие_Матери,Наличие_Отца,Опекунство,Село,...,male,female,sex_summ,birth_year_int,basis,language,country,region,city,parents_country
ind,,,,,,,,,,,,,,,,,,,,,
0,72716,16019,2015,2014.0,0.0,0.0,0,0.0,0.0,0.0,...,0,1,1,1998,2,1,8,14,74,10
1,63306,14895,2010,0.0,0.0,0.0,1,1.0,0.0,0.0,...,1,0,1,1992,4,0,8,14,74,10


## Создание тренировочного и тестового датафремов

In [11]:
train, test = train_test_split(frame_to_train, test_size=0.20)

In [12]:
train_input = train.drop('Статус', axis=1)
train_output = train['Статус']


In [13]:
input_arr = train_input.to_numpy()
input_df_size = len(input_arr)

In [ ]:
output = np.ones((1,input_df_size))
output[0] = train_output.to_numpy()
output_arr = output.reshape(input_df_size,1)
output_arr

In [14]:
test_input = test.drop('Статус', axis=1)
test_output = test['Статус']

In [15]:
test_arr = test_input.to_numpy()
test_arr_size = len(test_arr)

In [16]:
test = np.ones((1,test_arr_size))
test[0] = test_output.to_numpy()
test_out = test.reshape(test_arr_size,1)
test_out

array([[3.],
       [3.],
       [4.],
       ...,
       [4.],
       [4.],
       [3.]])

# SKlearn модель обучения

In [17]:

clf = RandomForestClassifier(random_state=0)

In [18]:
train_input.head(2)

,ID,Код_группы,Год_Поступления,Год_Окончания_УЗ,Пособие,Общежитие,Наличие_Матери,Наличие_Отца,Опекунство,Село,...,male,female,sex_summ,birth_year_int,basis,language,country,region,city,parents_country
ind,,,,,,,,,,,,,,,,,,,,,
7018,67671,17081,2014,2014.0,0.0,0.0,0,0.0,0.0,0.0,...,0,1,1,1998,3,2,8,14,417,10
7577,68062,19400,2014,2014.0,0.0,0.0,0,0.0,0.0,0.0,...,0,1,1,1996,2,1,8,14,74,10


In [19]:
clf.fit(train_input, train_output)

RandomForestClassifier(random_state=0)

In [21]:
pred = clf.predict(test_input)

In [22]:
test_output.head(3)

ind
6855     3
12391    3
1844     4
Name: Статус, dtype: int64

In [23]:
from sklearn.metrics import f1_score
f1_score(test_output, pred, average='macro', zero_division = 0)

0.728882309587109

In [24]:
df_test_base = pd.read_csv("data/test_frame.csv")
df_submission = pd.read_csv("data/sample_submission.csv")

FileNotFoundError: [Errno 2] No such file or directory: &#39;data/sample_submission.csv&#39;

In [ ]:
df_test_base.columns

In [ ]:
df_test = df_test_base.drop('Unnamed: 0', axis=1)
df_test = df_test.drop('ID', axis=1)
df_test

In [ ]:
df_test = df_test.drop('Статус', axis=1)

In [ ]:
df_test = df_test.fillna(0)

In [ ]:
df_test_pred = clf.predict(df_test)

In [ ]:
df_test_base["Статус"] = df_test_pred
df_test_base.head(4)

In [ ]:
df_submission

In [ ]:
make_csv = df_test_base.loc[:, ['ID', 'Статус']]
#make_csv['reg'] = np.arange(0, make_csv.shape[0])
make_csv

In [ ]:
id_hum = 64996
make_csv.loc[make_csv['ID'] == id_hum]

In [ ]:
for i in range(df_submission.shape[0]):
    id_hum = df_submission.loc[i, ['ID']]
    #print(id_hum)
    status = make_csv.loc[make_csv['ID'] == id_hum.values[0], 'Статус']
    try:
        df_submission.loc[i, ['Статус']] = status.values[0]
    except:
        df_submission.loc[i, ['Статус']] = 4
    print(df_submission.loc[i, ['Статус']])

In [ ]:
df_submission.to_csv('data/result/sample_submission.csv', index=False)

In [ ]:
np.arange(0, 10)

In [ ]:
make_csv.isna().any().all()

In [ ]:
df_submission.groupby('Статус').count()